In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier

from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay

In [ ]:
file_path = 'C:\Users\DELL\OneDrive\Documents\data\Project_Intrusion_Detection\P20_Intrusion_Detection\Intrusion Detection\\NSL_KDD_Train.csv'
file_path_test = 'C:\Users\DELL\OneDrive\Documents\data\Project_Intrusion_Detection\P20_Intrusion_Detection\Intrusion Detection\\NSL_KDD_Test.csv'

In [7]:
df = pd.read_csv(file_path)
test_df = pd.read_csv(file_path_test)

In [8]:
# add the column labels
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
])

df.columns = columns
test_df.columns = columns


df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal
1,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune
2,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal
3,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal
4,0,tcp,private,REJ,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune


In [9]:
test_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,tcp,private,REJ,0,0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
1,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal
2,0,icmp,eco_i,SF,20,0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint
3,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,86,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan
4,0,tcp,http,SF,267,14515,0,0,0,0,...,255,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal


In [10]:
df.shape

(125972, 42)

In [11]:
test_df.shape

(22543, 42)

In [12]:
df['attack'].value_counts()

normal             67342
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: attack, dtype: int64

In [13]:
test_df['attack'].value_counts()

normal             9711
neptune            4656
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess           331
saint               319
mailbomb            293
snmpgetattack       178
portsweep           157
ipsweep             141
httptunnel          133
nmap                 73
pod                  41
buffer_overflow      20
multihop             18
named                17
ps                   15
sendmail             14
rootkit              13
xterm                13
teardrop             12
xlock                 9
land                  7
xsnoop                4
ftp_write             3
worm                  2
loadmodule            2
perl                  2
sqlattack             2
udpstorm              2
phf                   2
imap                  1
Name: attack, dtype: int64

In [14]:
dos_attacks = ['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm']
probe_attacks = ['ipsweep','mscan','nmap','portsweep','saint','satan']
privilege_attacks = ['buffer_overflow','loadmdoule','perl','ps','rootkit','sqlattack','xterm']
access_attacks = ['ftp_write','guess_passwd','http_tunnel','imap','multihop','named','phf','sendmail','snmpgetattack','snmpguess','spy','warezclient','warezmaster','xclock','xsnoop']

In [15]:
df['attack_category'] = df['attack'].apply(
    lambda x: 'dos' if x in dos_attacks else (
        'probe' if x in probe_attacks else (
            'privilege' if x in privilege_attacks else (
                'access' if x in access_attacks else 'normal'
            )
        )
    )
)

test_df['attack_category'] = test_df['attack'].apply(
    lambda x: 'dos' if x in dos_attacks else (
        'probe' if x in probe_attacks else (
            'privilege' if x in privilege_attacks else (
                'access' if x in access_attacks else 'normal'
            )
        )
    )
)

In [16]:
df['attack_category'].value_counts()

normal       67351
dos          45927
probe        11656
access         995
privilege       43
Name: attack_category, dtype: int64

In [17]:
test_df['attack_category'].value_counts()

normal       9855
dos          7459
access       2743
probe        2421
privilege      65
Name: attack_category, dtype: int64

In [18]:
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,attack_category
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,normal
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,dos
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,normal
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,normal
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune,dos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,dos
125968,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,normal
125969,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,normal
125970,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,dos


In [19]:
test_df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,attack_category
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,dos
1,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal,normal
2,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint,probe
3,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan,probe
4,0,tcp,http,SF,267,14515,0,0,0,0,...,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0,tcp,smtp,SF,794,333,0,0,0,0,...,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal,normal
22539,0,tcp,http,SF,317,938,0,0,0,0,...,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal,normal
22540,0,tcp,http,SF,54540,8314,0,0,0,2,...,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back,dos
22541,0,udp,domain_u,SF,42,42,0,0,0,0,...,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal,normal


In [20]:
df.drop(['attack'],axis=1,inplace=True)
test_df.drop(['attack'],axis=1,inplace=True)

In [21]:
df.shape

(125972, 42)

In [22]:
test_df.shape

(22543, 42)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125972 entries, 0 to 125971
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125972 non-null  int64  
 1   protocol_type                125972 non-null  object 
 2   service                      125972 non-null  object 
 3   flag                         125972 non-null  object 
 4   src_bytes                    125972 non-null  int64  
 5   dst_bytes                    125972 non-null  int64  
 6   land                         125972 non-null  int64  
 7   wrong_fragment               125972 non-null  int64  
 8   urgent                       125972 non-null  int64  
 9   hot                          125972 non-null  int64  
 10  num_failed_logins            125972 non-null  int64  
 11  logged_in                    125972 non-null  int64  
 12  num_compromised              125972 non-null  int64  
 13 

In [24]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 0 to 22542
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     22543 non-null  int64  
 1   protocol_type                22543 non-null  object 
 2   service                      22543 non-null  object 
 3   flag                         22543 non-null  object 
 4   src_bytes                    22543 non-null  int64  
 5   dst_bytes                    22543 non-null  int64  
 6   land                         22543 non-null  int64  
 7   wrong_fragment               22543 non-null  int64  
 8   urgent                       22543 non-null  int64  
 9   hot                          22543 non-null  int64  
 10  num_failed_logins            22543 non-null  int64  
 11  logged_in                    22543 non-null  int64  
 12  num_compromised              22543 non-null  int64  
 13  root_shell      

# ENCODING

In [25]:
lab = LabelEncoder()
df['protocol_type'] = lab.fit_transform(df['protocol_type'])
df['service'] = lab.fit_transform(df['service'])
df['flag'] = lab.fit_transform(df['flag'])
df['attack_category'] = lab.fit_transform(df['attack_category'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125972 entries, 0 to 125971
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125972 non-null  int64  
 1   protocol_type                125972 non-null  int32  
 2   service                      125972 non-null  int32  
 3   flag                         125972 non-null  int32  
 4   src_bytes                    125972 non-null  int64  
 5   dst_bytes                    125972 non-null  int64  
 6   land                         125972 non-null  int64  
 7   wrong_fragment               125972 non-null  int64  
 8   urgent                       125972 non-null  int64  
 9   hot                          125972 non-null  int64  
 10  num_failed_logins            125972 non-null  int64  
 11  logged_in                    125972 non-null  int64  
 12  num_compromised              125972 non-null  int64  
 13 

In [26]:
test_df['protocol_type'] = lab.fit_transform(test_df['protocol_type'])
test_df['service'] = lab.fit_transform(test_df['service'])
test_df['flag'] = lab.fit_transform(test_df['flag'])
test_df['attack_category'] = lab.fit_transform(test_df['attack_category'])

test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 0 to 22542
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     22543 non-null  int64  
 1   protocol_type                22543 non-null  int32  
 2   service                      22543 non-null  int32  
 3   flag                         22543 non-null  int32  
 4   src_bytes                    22543 non-null  int64  
 5   dst_bytes                    22543 non-null  int64  
 6   land                         22543 non-null  int64  
 7   wrong_fragment               22543 non-null  int64  
 8   urgent                       22543 non-null  int64  
 9   hot                          22543 non-null  int64  
 10  num_failed_logins            22543 non-null  int64  
 11  logged_in                    22543 non-null  int64  
 12  num_compromised              22543 non-null  int64  
 13  root_shell      

In [27]:
# Retrieve the mapping of encoded labels to original categories
mapping = {index: category for index, category in enumerate(lab.classes_)}

# Print the mapping
print(mapping)

{0: 'access', 1: 'dos', 2: 'normal', 3: 'privilege', 4: 'probe'}


In [28]:
df['attack_category'].value_counts()

2    67351
1    45927
4    11656
0      995
3       43
Name: attack_category, dtype: int64

In [29]:
test_df['attack_category'].value_counts()

2    9855
1    7459
0    2743
4    2421
3      65
Name: attack_category, dtype: int64

# FEATURE SELECTION

In [30]:
df.corr()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_category
duration,1.000000,0.038241,0.092857,-0.063389,0.070737,0.034878,-0.001553,-0.009866,0.003830,0.000705,...,-0.109777,-0.116006,0.254195,0.228737,-0.026670,-0.064949,-0.064361,0.173815,0.199023,0.193406
protocol_type,0.038241,1.000000,0.029993,0.093669,-0.000974,-0.000608,-0.001757,0.169534,-0.000965,-0.011857,...,0.103918,0.001701,0.131380,-0.209105,-0.356183,-0.079399,-0.077926,-0.015435,-0.046938,-0.036562
service,0.092857,0.029993,1.000000,-0.304011,-0.001631,0.003596,-0.009952,0.084404,0.010980,-0.064067,...,-0.407702,-0.452702,0.284071,-0.111163,-0.156213,0.281632,0.277592,0.150666,0.148403,-0.087216
flag,-0.063389,0.093669,-0.304011,1.000000,-0.008114,-0.004096,-0.010373,0.067215,0.005811,0.068438,...,0.582695,0.630126,-0.283606,0.195689,0.073775,-0.443438,-0.443223,-0.683310,-0.718778,0.246908
src_bytes,0.070737,-0.000974,-0.001631,-0.008114,1.000000,0.000204,-0.000109,-0.000693,-0.000059,0.000295,...,-0.006861,-0.006572,0.000900,0.000431,-0.001655,-0.004503,-0.003397,-0.001468,0.012449,0.014961
dst_bytes,0.034878,-0.000608,0.003596,-0.004096,0.000204,1.000000,-0.000069,-0.000440,0.000248,-0.000344,...,-0.004224,-0.004424,0.011031,0.011747,-0.001281,-0.003024,-0.002944,0.011729,0.011223,0.010492
land,-0.001553,-0.001757,-0.009952,-0.010373,-0.000109,-0.000069,1.000000,-0.001261,-0.000109,-0.001340,...,-0.014159,0.011597,-0.004516,0.033851,0.070474,0.019840,0.012276,-0.005222,-0.005303,-0.008666
wrong_fragment,-0.009866,0.169534,0.084404,0.067215,-0.000693,-0.000440,-0.001261,1.000000,-0.000692,-0.008508,...,-0.045241,-0.048734,0.059796,0.037177,-0.016252,-0.051917,-0.055918,0.028890,-0.033682,-0.084395
urgent,0.003830,-0.000965,0.010980,0.005811,-0.000059,0.000248,-0.000109,-0.000692,1.000000,0.000293,...,-0.007897,-0.004489,0.006840,0.002741,0.005176,-0.004749,-0.004834,-0.002999,-0.002912,-0.001470
hot,0.000705,-0.011857,-0.064067,0.068438,0.000295,-0.000344,-0.001340,-0.008508,0.000293,1.000000,...,-0.051865,-0.036294,-0.012293,-0.034536,-0.024715,-0.058223,-0.058215,-0.030555,-0.031670,-0.057510


In [31]:
corr = df.corr()['attack_category']
corr = corr.drop(['attack_category'])

# Set thresholds for correlation coefficients
high_threshold = 0.5  # Threshold for highly correlated features
low_threshold = 0.1   # Threshold for low correlated features

# Find highly correlated features
highly_correlated_features = corr[abs(corr) > high_threshold].index.tolist()

# Find low correlated features
low_correlated_features = corr[abs(corr) < low_threshold].index.tolist()

In [32]:
print(highly_correlated_features)
print(low_correlated_features)

['serror_rate', 'srv_serror_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate']
['protocol_type', 'service', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'is_host_login', 'is_guest_login', 'srv_count']


In [33]:
# Combine highly correlated and low correlated features
selected_features = highly_correlated_features + low_correlated_features

In [34]:
selected_features

['serror_rate',
 'srv_serror_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'protocol_type',
 'service',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'is_host_login',
 'is_guest_login',
 'srv_count']

In [35]:
train_data = df.drop(columns=selected_features)
test_data = df.drop(columns=selected_features)

In [36]:
# # Remove highly correlated features from both train and test data
# train_data = df.drop(columns=highly_correlated_features)
# test_data= test_df.drop(columns=highly_correlated_features)

In [532]:
train_data.shape

(125972, 19)

In [533]:
test_data.shape

(125972, 19)

spliting X & y

In [534]:
X_train = train_data.iloc[:,:-1]
y_train = train_data.iloc[:,-1]

X_test = test_data.iloc[:,:-1]
y_test = test_data.iloc[:,-1]

# Feature Scaling

In [535]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(125972, 18)
(125972, 18)
(125972,)
(125972,)


In [536]:
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.fit_transform(X_test)

# SVM

In [537]:
svm = SVC()

In [538]:
svm.fit(X_scaled_train,y_train)

SVC()

In [539]:
y_pred = svm.predict(X_scaled_test)

In [540]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.50      0.64       995
           1       0.99      0.98      0.98     45927
           2       0.97      0.99      0.98     67351
           3       0.00      0.00      0.00        43
           4       0.99      0.96      0.97     11656

    accuracy                           0.98    125972
   macro avg       0.77      0.68      0.72    125972
weighted avg       0.98      0.98      0.98    125972



c:\Project\intrusion_detection\env1\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Project\intrusion_detection\env1\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Project\intrusion_detection\env1\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [541]:
print(confusion_matrix(y_test,y_pred))

[[  494     1   496     0     4]
 [    0 44970   951     0     6]
 [   49   478 66714     0   110]
 [    1     4    38     0     0]
 [    0    38   470     0 11148]]


HyperParametric Tuning

In [41]:
param_grid = {
    'C': [0.1, 1, 10],  # Regularization parameter
    'gamma': [0.1, 1, 10],  # Kernel coefficient
    'kernel': ['rbf', 'linear']  # Kernel type
}
svm_clf = SVC(random_state=42)

In [ ]:
# Reduce the search space and enable parallelization with n_jobs
clf = GridSearchCV(svm_clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
clf.fit(X_train, y_train)

# Printing the best parameters
print("Best Parameters:", clf.best_params_)

# ANN

In [542]:
from keras.utils import to_categorical
y_train_cat=to_categorical(y_train)
y_test_cat=to_categorical(y_test)

In [543]:
num_classes = 5  

ann = Sequential()
ann.add(Dense(X_train.shape[1], activation='relu'))
ann.add(Dense(X_train.shape[1], activation='relu'))
ann.add(Dense(X_train.shape[1], activation='relu'))
ann.add(Dense(X_train.shape[1], activation='relu'))
ann.add(Dense(num_classes, activation='softmax'))

In [544]:
ann.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [545]:
ann.fit(X_scaled_train,y_train_cat,batch_size=50,epochs=50)

Epoch 1/50


2520/2520 [==============================] - 6s 2ms/step - loss: 0.1666 - accuracy: 0.9506
Epoch 2/50
2520/2520 [==============================] - 4s 2ms/step - loss: 0.0790 - accuracy: 0.9732
Epoch 3/50
2520/2520 [==============================] - 4s 1ms/step - loss: 0.0664 - accuracy: 0.9770
Epoch 4/50
2520/2520 [==============================] - 4s 1ms/step - loss: 0.0601 - accuracy: 0.9792
Epoch 5/50
2520/2520 [==============================] - 4s 2ms/step - loss: 0.0564 - accuracy: 0.9805
Epoch 6/50
2520/2520 [==============================] - 4s 2ms/step - loss: 0.0536 - accuracy: 0.9812
Epoch 7/50
2520/2520 [==============================] - 4s 1ms/step - loss: 0.0520 - accuracy: 0.9820
Epoch 8/50
2520/2520 [==============================] - 4s 2ms/step - loss: 0.0500 - accuracy: 0.9822
Epoch 9/50
2520/2520 [==============================] - 4s 2ms/step - loss: 0.0487 - accuracy: 0.9830
Epoch 10/50
2520/2520 [==============================] - 4s 2ms/step - loss: 0.0

In [546]:
loss,accuracy = ann.evaluate(X_scaled_test,y_test_cat)
print('Loss',loss)
print('Accuracy',accuracy)

  32/3937 [..............................] - ETA: 7s - loss: 0.0257 - accuracy: 0.9912   

3937/3937 [==============================] - 6s 1ms/step - loss: 0.0306 - accuracy: 0.9883
Loss 0.030606592074036598
Accuracy 0.9883227944374084


In [547]:
y_pred=ann.predict(X_scaled_test)
y_pred

   1/3937 [..............................] - ETA: 6:57

3937/3937 [==============================] - 5s 1ms/step


array([[4.7051559e-18, 3.1944815e-11, 1.0000000e+00, 2.1911382e-13,
        3.1476222e-10],
       [0.0000000e+00, 1.0000000e+00, 6.7370841e-15, 1.1657253e-30,
        5.0562332e-13],
       [3.0948495e-09, 3.3242691e-07, 9.9999845e-01, 3.2947977e-08,
        1.1384626e-06],
       ...,
       [8.8290870e-03, 3.0684400e-06, 9.9017274e-01, 9.6596737e-04,
        2.9002204e-05],
       [0.0000000e+00, 1.0000000e+00, 3.3824133e-16, 1.1343711e-33,
        3.0720631e-14],
       [1.0079829e-01, 9.0775475e-06, 8.9883929e-01, 1.7749758e-04,
        1.7586656e-04]], dtype=float32)

In [548]:
y_test_cat

array([[0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [549]:
print(classification_report(y_test_cat.argmax(axis=1),y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.85      0.67      0.75       995
           1       0.99      0.99      0.99     45927
           2       0.99      0.99      0.99     67351
           3       0.00      0.00      0.00        43
           4       0.99      0.98      0.99     11656

    accuracy                           0.99    125972
   macro avg       0.76      0.73      0.74    125972
weighted avg       0.99      0.99      0.99    125972



c:\Project\intrusion_detection\env1\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Project\intrusion_detection\env1\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Project\intrusion_detection\env1\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [551]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_category
0,0,2,44,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,2
1,0,1,49,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,1
2,0,1,24,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,2
3,0,1,24,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,2
4,0,1,49,1,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,1


In [559]:
train_data.head()

,duration,flag,logged_in,num_outbound_cmds,count,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_category
0,0,9,0,0,13,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.0,0.00,2
1,0,5,0,0,123,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,0.0,0.00,1
2,0,9,1,0,5,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.0,0.01,2
3,0,9,1,0,30,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.0,0.00,2
4,0,1,0,0,121,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,1.0,1.00,1


In [557]:
test_data.head()

,duration,flag,logged_in,num_outbound_cmds,count,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_category
0,0,9,0,0,13,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.0,0.00,2
1,0,5,0,0,123,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,0.0,0.00,1
2,0,9,1,0,5,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.0,0.01,2
3,0,9,1,0,30,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.0,0.00,2
4,0,1,0,0,121,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,1.0,1.00,1


In [567]:
X_test.loc[1].values

array([0.00e+00, 5.00e+00, 0.00e+00, 0.00e+00, 1.23e+02, 0.00e+00,
       0.00e+00, 5.00e-02, 7.00e-02, 0.00e+00, 2.55e+02, 2.60e+01,
       1.00e-01, 5.00e-02, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00])

In [568]:
y_new=svm.predict(scaler.transform([[0.00e+00, 5.00e+00, 0.00e+00, 0.00e+00, 1.23e+02, 0.00e+00,
       0.00e+00, 5.00e-02, 7.00e-02, 0.00e+00, 2.55e+02, 2.60e+01,
       1.00e-01, 5.00e-02, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00]]))
# ind=y_new.argmax()
# ind
y_new
# pred=lab.inverse_transform(y_new)
# print("Class",pred.item())

c:\Project\intrusion_detection\env1\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([1])

In [574]:
y_new2 = ann.predict(scaler.transform([[0.00e+00, 5.00e+00, 0.00e+00, 0.00e+00, 1.23e+02, 0.00e+00,
                        0.00e+00, 5.00e-02, 7.00e-02, 0.00e+00, 2.55e+02, 2.60e+01,
                        1.00e-01, 5.00e-02, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00]]))

# Find the index of the maximum value in the prediction array
ind = y_new.argmax()

# Now ind contains the index of the maximum value in the prediction array
ind

1/1 [==============================] - 0s 33ms/step


c:\Project\intrusion_detection\env1\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1